<a href="https://colab.research.google.com/github/rickyprimay/smart_waste_management_model/blob/main/smart_waste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import cv2
import numpy as np
import zipfile
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
!pip install tensorflowjs
!pip install TensorFlow==2.15.0
!pip install tensorflow-decision-forests==1.8.1

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tensorstore to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorstore to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached orbax_checkpoint-0.6.4-py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 8.4 MB/s eta 0:00:00
  

In [ ]:
import tensorflowjs as tfjs

In [ ]:
image_size = (64, 64)

In [ ]:
def load_images_from_folder(folder):
    images = []
    labels = []

    for category in ['O', 'R']:
        category_path = os.path.join(folder, category)

        for filename in os.listdir(category_path):
            img_path = os.path.join(category_path, filename)
            img = cv2.imread(img_path)

            if img is not None:
                img = cv2.resize(img, image_size)
                img = img.astype('float32') / 255.0
                images.append(img)
                labels.append(0 if category == 'O' else 1)

    return np.array(images), np.array(labels)

In [ ]:
zip_file_path = '/content/archive.zip'
extract_folder = '/content/extracted_data'
os.makedirs(extract_folder, exist_ok=True)

In [ ]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

In [ ]:
train_dir = '/content/extracted_data/DATASET/TRAIN'
train_images, train_labels = load_images_from_folder(train_dir)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [ ]:
input_shape = (64, 64, 3)

In [ ]:
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(64, 64, 3)),\
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
565/565 [==============================] - 16s 27ms/step - loss: 0.6918 - accuracy: 0.7477 - val_loss: 0.6730 - val_accuracy: 0.6592
Epoch 2/10
565/565 [==============================] - 16s 29ms/step - loss: 0.5577 - accuracy: 0.7667 - val_loss: 0.5348 - val_accuracy: 0.7456
Epoch 3/10
565/565 [==============================] - 24s 42ms/step - loss: 0.4984 - accuracy: 0.7776 - val_loss: 0.4551 - val_accuracy: 0.8043
Epoch 4/10
565/565 [==============================] - 15s 27ms/step - loss: 0.4810 - accuracy: 0.7908 - val_loss: 0.4486 - val_accuracy: 0.7977
Epoch 5/10
565/565 [==============================] - 15s 27ms/step - loss: 0.4611 - accuracy: 0.7993 - val_loss: 0.4434 - val_accuracy: 0.8001
Epoch 6/10
565/565 [==============================] - 15s 26ms/step - loss: 0.4536 - accuracy: 0.7982 - val_loss: 0.4672 - val_accuracy: 0.7886
Epoch 7/10
565/565 [==============================] - 15s 27ms/step - loss: 0.4552 - accuracy: 0.7998 - val_loss: 0.4467 - val_accuracy:

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
tflite_model_path = '/content/model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
model.save('/content/model.keras')
tf.keras.backend.clear_session()
model = tf.keras.models.load_model('/content/model.keras')
model.summary

<bound method Model.summary of <tf_keras.src.engine.sequential.Sequential object at 0x7c7f29e97be0>>

In [ ]:
tfjs_model_path = '/content/model_tfjsasaseras'
tfjs.converters.save_keras_model(model, tfjs_model_path)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 dense (Dense)               (None, 128)               1572992   
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1573250 (6.00 MB)
Trainable params: 1573250 (6.00 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
uploaded = files.upload()
for filename in uploaded.keys():
    img = cv2.imread(filename)
    if img is not None:
        img = cv2.resize(img, image_size)
        img = img.astype('float32') / 255.0
        img = img.reshape(1, image_size[0], image_size[1], 3)

        interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
        interpreter.allocate_tensors()

        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        interpreter.set_tensor(input_details[0]['index'], img)
        interpreter.invoke()

        output_data = interpreter.get_tensor(output_details[0]['index'])
        prediction = np.argmax(output_data, axis=1)
        label = 'Organic' if prediction[0] == 0 else 'Recyclable'
        print(f"Predicted Label for {filename}: {label}")

Saving O_2.jpg to O_2.jpg
Predicted Label for O_2.jpg: Organic
